In [1]:
library(Seurat)

Warning message:
“package ‘Seurat’ was built under R version 3.6.3”


In [2]:
mat <- list()
mat$HC1 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475048_C51_filtered_feature_bc_matrix.h5")
mat$HC2 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475049_C52_filtered_feature_bc_matrix.h5")
mat$HC3 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475050_C100_filtered_feature_bc_matrix.h5")
mat$HC4 <- Read10X(data.dir = "/fdata/single-cell/COVID-19/GSM3660650/")
mat$M1 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339769_C141_filtered_feature_bc_matrix.h5")
mat$M2 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339770_C142_filtered_feature_bc_matrix.h5")
mat$M3 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339772_C144_filtered_feature_bc_matrix.h5")
mat$S1 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339773_C145_filtered_feature_bc_matrix.h5")
mat$S2 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339771_C143_filtered_feature_bc_matrix.h5")
mat$S3 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4339774_C146_filtered_feature_bc_matrix.h5")
mat$S4 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475051_C148_filtered_feature_bc_matrix.h5")
mat$S5 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475052_C149_filtered_feature_bc_matrix.h5")
mat$S6 <- Read10X_h5(filename = "/fdata/single-cell/COVID-19/GSM4475053_C152_filtered_feature_bc_matrix.h5")
meta.data <- read.delim("/fdata/single-cell/COVID-19/all.cell.annotation.meta.txt")
rownames(meta.data) <- meta.data$ID
head(meta.data)

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in spa

,ID,sample,sample_new,group,disease,hasnCoV,cluster,celltype
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>
AAACCTGAGACACTAA_1,AAACCTGAGACACTAA_1,C51,HC1,HC,N,N,3,Macrophages
AAACCTGAGGAGTACC_1,AAACCTGAGGAGTACC_1,C51,HC1,HC,N,N,3,Macrophages
AAACCTGAGGATATAC_1,AAACCTGAGGATATAC_1,C51,HC1,HC,N,N,3,Macrophages
AAACCTGAGGTCATCT_1,AAACCTGAGGTCATCT_1,C51,HC1,HC,N,N,3,Macrophages
AAACCTGCACGGATAG_1,AAACCTGCACGGATAG_1,C51,HC1,HC,N,N,5,Macrophages
AAACCTGCAGGGAGAG_1,AAACCTGCAGGGAGAG_1,C51,HC1,HC,N,N,2,Macrophages


In [3]:
numbering = c(HC1 = 1,
              HC2 = 2,
              HC3 = 3,
              HC4 = 4,
              M1 = 5,
              M2 = 6,
              M3 = 7,
              S1 = 9,
              S2 = 8,
              S3 = 10,
              S4 = 11,
              S5 = 12,
              S6 = 13)

In [4]:
for (i in names(mat)){
  colnames(mat[[i]]) <- gsub('-', '_', colnames(mat[[i]]))
  colnames(mat[[i]]) <- gsub('1', numbering[i], colnames(mat[[i]]))
}

In [5]:
for (i in names(mat)){
 mat[[i]] <- mat[[i]][, meta.data$ID[meta.data$sample_new == i]]
}

In [6]:
objs <- list()
for (i in names(mat)){
 objs[[i]] <- CreateSeuratObject(mat[[i]], project = i, meta.data = meta.data[meta.data$sample_new == i, ])
}

In [7]:
obj <- merge(objs[[1]], objs[-1])

In [8]:
obj <- NormalizeData(obj, verbose=FALSE)
obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000, verbose=FALSE)



In [10]:
dir.create("data/normalized/covid/data/", showWarnings = FALSE, recursive = TRUE)
dir.create("data/normalized/covid/cell_labels/", showWarnings = FALSE, recursive = TRUE)

dir.create("data/normalized/covid_generalization/data/", showWarnings = FALSE, recursive = TRUE)
dir.create("data/normalized/covid_generalization/cell_labels/", showWarnings = FALSE, recursive = TRUE)

initial_batches <- c('HC1', 'HC2', 'HC3')

covid.list <- SplitObject(obj, split.by = "sample_new")
common_variable_features = VariableFeatures(obj)

for (i in 1:length(x = covid.list)) {
    data_to_save <- as.data.frame(covid.list[[i]]$RNA@data[common_variable_features,])
    tech_name <- unique(covid.list[[i]]$sample_new)
    
    write.csv(data_to_save, paste("data/normalized/covid_generalization/data/", tech_name, ".csv", sep=""))
    write.csv(covid.list[[i]]$celltype, paste("data/normalized/covid_generalization/cell_labels/", tech_name, ".csv", sep=""))
    
    if (tech_name %in% initial_batches) {
        write.csv(data_to_save, paste("data/normalized/covid/data/", tech_name, ".csv", sep=""))
        write.csv(covid.list[[i]]$celltype, paste("data/normalized/covid/cell_labels/", tech_name, ".csv", sep=""))
    }
}

In [ ]:
obj <- ScaleData(obj, verbose=FALSE)
obj <- RunPCA(obj, features = VariableFeatures(object = obj), verbose=FALSE)
ElbowPlot(obj)

In [ ]:
obj <- RunUMAP(obj, dims = 1:30, verbose=FALSE)

In [ ]:
DimPlot(obj, reduction = "umap", group.by = "celltype", label = T, pt.size = 0.01)
DimPlot(obj, reduction = "umap", group.by = "group")
DimPlot(obj, reduction = "umap", group.by = "disease")